# 파이썬 크롤링 라이브러리

## 파이썬 표준 라이브러리 urllib 사용하기

### 웹페이지 다운로드

In [1]:
import urllib
def download(url):
    return urllib.request.urlopen(url)

urlopen()함수를 통해 해당 서버에서 요청받은 ("HTTPResponse")객체를 반환 합니다.  
이 함수의 문제점은 웹페이지를 다운로드할 때 제어할 수 없는 에러가 발생할 수 있다는 점이에요.

In [2]:
from urllib.error import URLError, HTTPError, ContentTooShortError

def download(url):
    try:
        html = urllib.request.urlopen(url)
    except (URLError, HTTPError, ContentTooShortError) as e:
        print('Download error', e.reason)
        html = None
    return html

download('https://www.google.com')

예외가 발생하는 경우를 확인하여 예외처리 코드를 추가시켜줍니다.

In [3]:
def download(url):
    try:
        html = urllib.request.urlopen(url).read()
    except (URLError, HTTPError, ContentTooShortError) as e:
        print('Download error', e.reason)
        html = None
    return html

### 웹페이지 분석하기

웹 페이지를 분석하기 위해선 HTML 사이의 데이터를 파싱해서 가져와야 합니다.  
그리고 이 데이터를 가져오는 방법은 urlopen()의 read()메소드를 이용하는 거에요.

In [4]:
def download(url):
    try:
        html = urllib.request.urlopen(url).read()
    except (URLError, HTTPError, ContentTooShortError) as e:
        print('Download error', e.reason)
        html = None
    return html

In [5]:
download('https://www.google.com')

b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ko"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="gbMaAqYzKEHXl9+lLj0auw==">(function(){window.google={kEI:\'n8c8X8rAGaa1mAWFx7ygBA\',kEXPI:\'0,202123,3,4,32,4,30,1151550,5663,731,223,756,4349,206,3204,10,1226,364,1499,611,206,383,246,5,1354,648,653,1661,1138,314,3,507,36,507,91,41,152,894,86,95,9,164,302,291,247,1119927,1197721,329546,13677,4855,32691,15248,867,17444,11240,9188,8384,4858,1362,9291,3022,4745,6,11027,1808,4020,978,7931,5297,2054,920,873,1217,9405,14528,234,4283,1396,1383,917,2277,8,2796,1593,1279,2212,239,291,149,1103,840,517,1522,4258,312,1137,2,2063,606,2025,2295,1947,245,1984,93,328,1284,16,2927,2246,1813,1787,2273,1,953,1989,856,7,6068,6285,4456,641,2449,3685,1742,4929,108,2855,552,908,2,971,684,1900,2397,7468,2179,1098,3,346,230,970,

자 이 데이터를 어떻게 파싱할까요? 문자열 함수..정규표현식을 이용하면 되는데.. 이 2가지만 가지고 분석하기에는…네 너무 힘듭니다.

## 파이썬 라이브러리 BeautifulSoup, Requests 사용하기

### 웹페이지 다운로드

requests의 get()메소드를 이용하여 웹페이지를 다운로드 받을 수 있습니다.

In [6]:
import requests
url = 'http://www.google.com'
response = requests.get(url)
response

<Response [200]>

STEP1-1에서 HTTP의 Response 관련 메소드에 2xx는 응답관련 메소드라고 했습니다. 200은 응답에 성공이란 뜻이에요.  
응답에 성공하면, 이어서 get()메소드의 text 멤버변수에 접근 할 수 있는데요, text는 HTML을 반환합니다. 다음은 예외처리를 한 코드에요

In [7]:
def download2(url):
    try:
        response = requests.get(url)
        html = response.text
    except requests.ConnectionError:
        print('Connection error')
        html = None
    return html

download2('https://www.google.com')

'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ko"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="EU9/xo/DILKcNJ0RQ2qk6A==">(function(){window.google={kEI:\'Gsg8X_zwF5jZz7sPxpa2wA8\',kEXPI:\'0,202123,3,4,32,4,30,570823,580727,5663,731,223,5104,207,3204,10,1226,364,1499,611,206,383,246,5,1015,339,648,3451,315,3,374,134,543,89,42,152,864,30,84,106,163,427,167,227,217,41,7,1119682,1197759,329508,13677,4855,32691,15248,861,17450,11240,9188,8384,4858,1362,9291,3021,2822,1924,8000,3033,1808,4020,978,7931,5297,2054,920,873,1217,2975,6430,7432,7095,4517,1399,1379,919,2279,7,2795,885,708,1279,2212,530,149,1103,840,517,1522,4258,109,203,1137,2,2063,606,2023,1777,520,1947,2229,93,328,1284,16,2927,2247,1812,1787,3227,2845,7,5599,469,6286,4455,641,2449,3685,1742,4929,108,3407,908,2,941,2614,2397,7470,2177,1098,3,57

### 웹페이지 분석하기

웹페이지 다운로드를 BeautifulSoup객체로 만들어 줍니다.

In [9]:
import requests
from bs4 import BeautifulSoup
html = requests.get('http://www.google.com')
soup = BeautifulSoup(html.text, 'html.parser')

그러면 html 태그를 클래스 변수를 이용해 접근할 수 있어요.

In [10]:
soup.html.body

<body bgcolor="#fff"><script nonce="m2gD0UaikQdZAFY26Fz4aw==">(function(){var src='/images/nav_logo229.png';var iesg=false;document.body.onload = function(){window.n && window.n();if (document.images){new Image().src=src;}
if (!iesg){document.f&&document.f.q.focus();document.gbqf&&document.gbqf.q.focus();}
}
})();</script><div id="mngb"><div id="gbar"><nobr><b class="gb1">검색</b> <a class="gb1" href="http://www.google.co.kr/imghp?hl=ko&amp;tab=wi">이미지</a> <a class="gb1" href="http://maps.google.co.kr/maps?hl=ko&amp;tab=wl">지도</a> <a class="gb1" href="https://play.google.com/?hl=ko&amp;tab=w8">Play</a> <a class="gb1" href="http://www.youtube.com/?gl=KR&amp;tab=w1">YouTube</a> <a class="gb1" href="https://news.google.com/?tab=wn">뉴스</a> <a class="gb1" href="https://mail.google.com/mail/?tab=wm">Gmail</a> <a class="gb1" href="https://drive.google.com/?tab=wo">드라이브</a> <a class="gb1" href="https://www.google.co.kr/intl/ko/about/products?tab=wh" style="text-decoration:none"><u>더보기</u> »</a><

예를들어, html이 태그 내용이 다음과 같다고 할께요,

In [11]:
soup = BeautifulSoup("<span>Wow it's so good!!</span>", 'html.parser')
soup.span

<span>Wow it's so good!!</span>

### find(), findAll()

BeautifulSoup에서 제공하는 이 두 함수는 정말 자주 사용되는 함수입니다. 태그에 더 쉽게 접근할 수 있고 태그의 속성에 따라 필터링도 할 수 있어요.  
BeautifulSoup객체로 만든 뒤 p태그 항목을 찾는 코드는 다음과 같아요.

In [12]:
# ''', """ 는 여러줄의 문자열을 입력할 때 사용
html='''<title>Fundamental</title> 
         <body>
          <p id='programming'>python</p> 
          <p id='programming'>java</p> 
          <p id='algorithm'>algorithm</p> 
          <p id='fundamental'>math</p> 
          <p id='programming'>C++</p> 
          </body>'''
soup = BeautifulSoup(html, 'html.parser')
soup.findAll({'p'})

[<p id="programming">python</p>,
 <p id="programming">java</p>,
 <p id="algorithm">algorithm</p>,
 <p id="fundamental">math</p>,
 <p id="programming">C++</p>]

p태그에서 속성값이 programming인 태그를 찾고 싶을때는 다음과 같이 해주면 됩니다.

In [13]:
soup.findAll('p', id='programming')

[<p id="programming">python</p>,
 <p id="programming">java</p>,
 <p id="programming">C++</p>]

# 네이버 환율 정보 크롤링 

In [14]:
import requests
url = 'https://m.stock.naver.com/marketindex/index.nhn'
response = requests.get(url)
response

<Response [200]>

In [15]:
def download3(url):
    try:
        response = requests.get(url)
        html = response.text
    except requests.ConnectionError:
        print('Connection error')
        html = None
    return html

download3('https://m.stock.naver.com/marketindex/index.nhn')

'\n\n\n\n\n<!-- header -->\n\n\n\n\n\n\n\n\n<!doctype html>\n\n<html lang="ko">\n<head>\n    <meta charset="utf-8">\n    <meta name="viewport" content="width=device-width,initial-scale=1.0,maximum-scale=1.0,minimum-scale=1.0,user-scalable=no">\n\n    \n    \n        \n        \n            <meta property="og:url" content="http://m.stock.naver.com/marketindex/index.nhn?menu=exchange"/>\n            \n                \n                    <meta property="og:title" content="네이버 증권"/>\n                \n                \n            \n            \n                \n                    <meta property="og:description" content="관심종목의 실시간 주가를 가장 빠르게 확인하는 곳"/>\n                \n                \n            \n            \n                \n                    <meta property="og:image" content="https://ssl.pstatic.net/static/m/stock/im/2016/08/og_stock-200.png"/>\n                \n                \n            \n        \n    \n\n    <meta property="og:type" content="article"/>\n    <meta pr

In [16]:
import requests
from bs4 import BeautifulSoup
html = requests.get('https://m.stock.naver.com/marketindex/index.nhn')
soup = BeautifulSoup(html.text, 'html.parser')

In [17]:
soup.html.body

<body class="marketindex">
<!-- u_skip -->
<div class="u_skip">
<a href="#content">본문 바로가기</a>
</div>
<header class="home_header" id="header" role="banner">
<!-- gnb -->
<div class="Ngnb">
<span class="Ngnb_logo"><a class="Nlogo_link" href="https://m.naver.com/"><span class="Nicon_logo">NAVER</span></a></span>
<div class="Ngnb_service">
<h1 class="Nservice_item"><a href="https://m.stock.naver.com/index.nhn"><span class="Nicon_service">증권</span></a></h1>
</div>
<div class="Ngnb_tool">
<a class="Ntool_button _btn_my" href="#" onclick="nclk(this, 'gnb.my', '', '');"><span class="Nicon_my">MY</span></a>
</div>
</div>
<div class="pop_mylist _my_layer">
</div>
<!-- 검색 -->
<!-- 모바일 -->
<div class="Nsearch">
<div class="Nsearch_inner">
<div class="Nsearch_box">
<div class="Nsearch_box_inner">
<form action="/searchItem.nhn" id="search" method="get" name="search" onsubmit="return delayed_submit(this)">
<span class="Nbox_text"><input class="Nbox_input_text" id="keyword_top" name="keyword_input" o

In [20]:
from bs4 import BeautifulSoup
import requests

#URL 가져오기
url = 'https://m.stock.naver.com/marketindex/index.nhn'
response = requests.get(url)

#Soup 객체 생성
soup = BeautifulSoup(response.text, 'html.parser')

#원하는 데이터 추출 - 국가
country = []
country_1 = soup.findAll('strong')
country_2 = soup.findAll('span', {'class':'stock_item'})
for c in country_1:
    country.append(c.text)
for c in country_2:
    country.append(c.text)
country.remove('컨텐츠 제공업체')
country.remove('증권')

#원하는 데이터 추출 - 환율정보
price = []
price_0 = soup.findAll('span', {"class":"stock_price"})
for p in price_0:
    price.append(p.text)

#데이터 정렬, (pandas)
import pandas as pd
data = {}
for i in range(len(country)):
    data[country[i]] = price[i]
    
pd.Series(data)

미국USD          1,181.50
유럽EUR          1,410.77
일본JPY          1,119.53
중국CNY            170.92
국제금            1,999.40
유가WTI             42.89
달러인덱스             92.25
남아프리카 ZAR         68.33
노르웨이 NOK         133.63
뉴질랜드 NZD         782.09
대만 TWD            40.19
덴마크 DKK          189.45
러시아 RUB           16.17
말레이지아 MYR        282.59
멕시코 MXN           53.36
몽골 MNT             0.41
미국 USD         1,181.50
바레인 BHD        3,134.12
방글라데시 BDT         13.93
베트남 VND 100        5.10
브라질 BRL          216.10
브루나이 BND         865.85
사우디 SAR          315.02
스웨덴 SEK          136.78
스위스 CHF        1,308.63
싱가포르 SGD         865.85
아랍에미 AED         321.66
영국 GBP         1,566.02
오만 OMR         3,068.83
요르단 JOD        1,666.43
유럽연합 EUR       1,410.77
이스라엘 ILS         347.47
이집트 EGP           74.21
인도 INR            15.79
인도네시아 IDR          8.01
일본 JPY 100     1,119.53
중국원화 CNY         170.92
체코 CZK            54.05
칠레 CLP             1.49
카자흐스탄 KZT          2.82
카타르 QAR          322.04
캐나다 CAD         